In [1778]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets,transforms
from torch.utils.data import DataLoader, TensorDataset

#  Download and Load Data

In [1779]:
transform = transforms.Compose([
    transforms.Resize((10, 10)),  # Resize images
    transforms.ToTensor(),          # Convert to Tensor
])

# Download and load the training dataset
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)

# Download and load the test dataset
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

print("Datasets downloaded successfully!")

Datasets downloaded successfully!


# Train, Test and Validation Split

In [1780]:
test_images = torch.stack([train_dataset[i][0] for i in range(3)])

# test_images = train_dataset[0][0].unsqueeze(0)

In [1781]:
test_images.shape

torch.Size([3, 1, 10, 10])

In [1782]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [1783]:
# Extract the first 5 images and their labels from the training dataset
x_train = train_dataset.data[:5].unsqueeze(1).float() / 255  # Normalize and add channel dimension
y_train = train_dataset.targets[:5]

# Extract the first 5 images and their labels from the test dataset
x_test = test_dataset.data[:5].unsqueeze(1).float() / 255  # Normalize and add channel dimension
y_test = test_dataset.targets[:5]

In [1784]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

(torch.Size([5, 1, 28, 28]),
 torch.Size([5]),
 torch.Size([5, 1, 28, 28]),
 torch.Size([5]))

# Convolutional Neural Network

In [1785]:
def relu(x):
  return torch.where(x < 0, 0, x)

def xavier_uniform(n_in, n_out):
  limit = torch.sqrt(torch.tensor(6.0) / (n_in + n_out))
  return torch.empty(n_in, n_out).uniform_(-limit, limit)

In [1786]:
def apply_padding(tensor, pad=1):
    # Get the shape of the original tensor
    original_batch, _, original_height, original_width = tensor.shape

    # Create a new tensor filled with zeros (padding value)
    padded_tensor = torch.zeros((original_batch, 1, original_height + 2 * pad, original_width + 2 * pad), dtype=tensor.dtype)

    # Place the original tensor in the center of the padded tensor
    padded_tensor[:, :, pad:pad + original_height, pad:pad + original_width] = tensor

    return padded_tensor

def rotate_180(tensor):
    return torch.flip(tensor, dims=[2, 3])

## Convolution

In [1787]:
def conv(a, kernel, stride=1, padding=0):

    a = apply_padding(a, pad=padding)

    batch, channels, rows, cols = a.shape
    ker_batch, ker_channels, ker_rows, ker_cols = kernel.shape


    # Calculate the output dimensions
    output_rows = (rows - ker_rows) // stride + 1
    output_cols = (cols - ker_cols) // stride + 1

    # Initialize an output tensor
    result = torch.zeros((batch, ker_batch, output_rows, output_cols), dtype=torch.float32)

    # Perform the convolution operation
    for b in range(batch):  # Iterate over the batch
        for k in range(ker_batch):  # Iterate over the output channels
            for i in range(output_rows):  # Iterate over the rows of the output
                for j in range(output_cols):  # Iterate over the columns of the output
                    row_start = i * stride
                    col_start = j * stride
                    sub_matrix = a[b, :, row_start:row_start + ker_rows, col_start:col_start + ker_cols]

                    # element-wise multiplication and summation
                    cov_op = (sub_matrix * kernel[k, :, :, :]).sum()
                    result[b, k, i, j] = cov_op

    return result

In [1788]:
def conv_back(a, kernel, stride=1, padding=0):

    a = apply_padding(a, pad=padding)

    batch, channels, rows, cols = a.shape
    ker_batch, ker_channels, ker_rows, ker_cols = kernel.shape


    # Calculate the output dimensions
    output_rows = (rows - ker_rows) // stride + 1
    output_cols = (cols - ker_cols) // stride + 1
    output_batch = (batch - ker_batch) // stride + 1

    # Initialize an output tensor
    result = torch.zeros((output_batch, ker_channels, output_rows, output_cols), dtype=torch.float32)

    # Perform the convolution operation

    for k in range(ker_batch):  # Iterate over the output channels
        for i in range(output_rows):  # Iterate over the rows of the output
            for j in range(output_cols):  # Iterate over the columns of the output
                cov_op = 0
                for b in range(batch):  # Iterate over the batch

                  row_start = i * stride
                  col_start = j * stride
                  sub_matrix = a[b, :, row_start:row_start + ker_rows, col_start:col_start + ker_cols]

                  # element-wise multiplication and summation
                  cov_op += (sub_matrix * kernel[b, :, :, :]).sum()
                  result[0, 0, i, j] = cov_op

    return result

## Testing Convolution

In [1789]:
a = torch.randn([2, 1, 6, 6])
a.shape

torch.Size([2, 1, 6, 6])

In [1790]:
kernel = torch.randn([2, 1, 4, 4])
kernel

tensor([[[[ 0.1682, -0.8352,  0.1929,  0.5968],
          [-0.0347,  0.4319, -0.6917, -1.0690],
          [-0.0359, -0.2369,  1.4733, -0.0764],
          [-0.2969,  0.0152,  0.3667,  0.7715]]],


        [[[ 0.1833, -1.6825,  1.8818, -1.6269],
          [ 1.2929, -1.3502, -1.0637, -0.6175],
          [ 0.0070,  1.5967, -0.6280,  1.0502],
          [-0.3086,  0.8544,  2.1633,  0.4656]]]])

In [1791]:
# Perform convolution using PyTorch

a_tor = a.clone()
kernel_tor = kernel.clone()

pytorch_conv = F.conv2d(a_tor, kernel_tor, stride=1)
print(pytorch_conv .shape)
print(pytorch_conv[0][0])

torch.Size([2, 2, 3, 3])
tensor([[-2.2798, -1.6293,  0.7451],
        [ 3.4613, -3.1083, -3.0839],
        [ 3.8613,  2.4461,  1.0630]])


In [1792]:
# Perform convolution
result = conv(a, kernel, stride=1)
print(result.shape)
print(result[0][0])

torch.Size([2, 2, 3, 3])
tensor([[-2.2798, -1.6293,  0.7451],
        [ 3.4613, -3.1083, -3.0839],
        [ 3.8613,  2.4461,  1.0630]])


## Max Pooling

In [1793]:
def max_pool(a, kernel, stride = kernel):

    batch, channels, rows, cols = a.shape

    # Calculate the output dimensions
    output_rows = (rows - kernel) // stride + 1
    output_cols = (cols - kernel) // stride + 1

    # Initialize an output tensor
    result = torch.zeros((batch, 1, output_rows, output_cols))

    # Perform the convolution operation
    for b in range(batch):
      for i in range(output_rows):
          for j in range(output_cols):
              row_start = i * stride
              col_start = j * stride
              sub_matrix = a[b, :,row_start:row_start+kernel, col_start:col_start+kernel]
              pool_op = sub_matrix.max()
              result[b, 0, i, j] = pool_op

    return result

In [1794]:
def pool_back(a, kernel, grad,  stride = kernel):

    batch, channels, rows, cols = a.shape

    # Calculate the output dimensions
    output_rows = (rows - kernel) // stride + 1
    output_cols = (cols - kernel) // stride + 1

    # Initialize an output tensor
    # result_identity = torch.zeros((batch, 1, rows, cols))
    result_gradient = torch.zeros((batch, 1, rows, cols))

    # Create an iterator by flattening the tensor
    grad_iterator = iter(grad.flatten())


    # Perform the convolution operation
    for b in range(batch):
      for i in range(output_rows):
          for j in range(output_cols):

              row_start = i * stride
              col_start = j * stride
              sub_matrix = a[b, :,row_start:row_start+kernel, col_start:col_start+kernel]
              # max_ele = sub_matrix.max()
              # Find the maximum value in the tensor
              max_ele = torch.max(sub_matrix)
              num_max_ele = (sub_matrix == max_ele).sum().item()

              _, sub_rows, sub_cols = sub_matrix.shape

              # Get the next element
              next_element = next(grad_iterator)

              for k in range(sub_rows):
                  for l in range(sub_cols):
                      if sub_matrix[0, k, l] == max_ele:
                          pool_op = (1 / num_max_ele)
                      else:
                          pool_op = 0

                      # result_identity[b, 0, row_start + k, col_start + l] = pool_op
                      result_gradient[b, 0, row_start + k, col_start + l] = pool_op * next_element

                      batch, channels, rows, cols = a.shape

    return result_gradient


## Testing Max Pooling

In [1795]:
b = test_images
b[0][0]

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0157, 0.0510, 0.1020, 0.1529, 0.1922, 0.0980,
         0.0000],
        [0.0000, 0.0000, 0.1176, 0.5216, 0.7059, 0.7647, 0.6000, 0.4431, 0.1686,
         0.0000],
        [0.0000, 0.0000, 0.0784, 0.4471, 0.7490, 0.3098, 0.1765, 0.0157, 0.0039,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0353, 0.4784, 0.3333, 0.0510, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0941, 0.5294, 0.5333, 0.0745, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0078, 0.1098, 0.4431, 0.8000, 0.1843, 0.0000,
         0.0000],
        [0.0000, 0.0118, 0.1059, 0.3647, 0.7216, 0.6902, 0.3294, 0.0353, 0.0000,
         0.0000],
        [0.0000, 0.1961, 0.6118, 0.6706, 0.4000, 0.0784, 0.0039, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0431, 0.0941, 0.0549, 0.0039, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000]])

In [1796]:
kernel = 2

In [1797]:
# Perform convolution using PyTorch

b_tor = b.clone()

pytorch_pool = F.max_pool2d(b_tor, kernel_size=kernel, stride=2)
print(pytorch_pool)

tensor([[[[0.0000, 0.0157, 0.1020, 0.1922, 0.0980],
          [0.0000, 0.5216, 0.7647, 0.6000, 0.1686],
          [0.0000, 0.0353, 0.5294, 0.5333, 0.0000],
          [0.0118, 0.3647, 0.7216, 0.8000, 0.0000],
          [0.1961, 0.6706, 0.4000, 0.0039, 0.0000]]],


        [[[0.0000, 0.0000, 0.2784, 0.4235, 0.0000],
          [0.0000, 0.3098, 0.8549, 0.8000, 0.0784],
          [0.0118, 0.6314, 0.2824, 0.6980, 0.1961],
          [0.0196, 0.6667, 0.5882, 0.4863, 0.0157],
          [0.0039, 0.5137, 0.4078, 0.0039, 0.0000]]],


        [[[0.0078, 0.0039, 0.0000, 0.0941, 0.0196],
          [0.4392, 0.1412, 0.0000, 0.5059, 0.0667],
          [0.5529, 0.4824, 0.4510, 0.6392, 0.0039],
          [0.0549, 0.1059, 0.0588, 0.5725, 0.0000],
          [0.0000, 0.0000, 0.0431, 0.4980, 0.0000]]]])


In [1798]:
# Perform pooling
result = max_pool(b, kernel, stride=2)

print(result)

tensor([[[[0.0000, 0.0157, 0.1020, 0.1922, 0.0980],
          [0.0000, 0.5216, 0.7647, 0.6000, 0.1686],
          [0.0000, 0.0353, 0.5294, 0.5333, 0.0000],
          [0.0118, 0.3647, 0.7216, 0.8000, 0.0000],
          [0.1961, 0.6706, 0.4000, 0.0039, 0.0000]]],


        [[[0.0000, 0.0000, 0.2784, 0.4235, 0.0000],
          [0.0000, 0.3098, 0.8549, 0.8000, 0.0784],
          [0.0118, 0.6314, 0.2824, 0.6980, 0.1961],
          [0.0196, 0.6667, 0.5882, 0.4863, 0.0157],
          [0.0039, 0.5137, 0.4078, 0.0039, 0.0000]]],


        [[[0.0078, 0.0039, 0.0000, 0.0941, 0.0196],
          [0.4392, 0.1412, 0.0000, 0.5059, 0.0667],
          [0.5529, 0.4824, 0.4510, 0.6392, 0.0039],
          [0.0549, 0.1059, 0.0588, 0.5725, 0.0000],
          [0.0000, 0.0000, 0.0431, 0.4980, 0.0000]]]])


## Data and Weights Initialization

In [1799]:
nhidden = 64
nhidden2 = 32
noutput = 10

In [1800]:
x_train.shape, y_train.shape

(torch.Size([5, 1, 28, 28]), torch.Size([5]))

In [1801]:

filter1 = torch.randn(1, 1, 3, 3) * 0.01
filter1 = filter1.requires_grad_(True)
filter2 = torch.randn(1, 1, 3, 3) * 0.01
filter2 = filter2.requires_grad_(True)
pool_kernel = 2

w1 = torch.randn(25, nhidden, requires_grad=True) * 0.01
b1 = torch.randn(1, nhidden, requires_grad=True) * 0.01

w2 = torch.randn(nhidden, nhidden2, requires_grad=True) * 0.01
b2 = torch.randn(1, nhidden2, requires_grad=True) * 0.01

w3 = torch.randn(nhidden2, noutput, requires_grad=True) * 0.01
b3 = torch.randn(1, noutput, requires_grad=True) * 0.01

## Forward Pass (Manual)

In [1802]:
# Convolution layers
conv1 = conv(x_train, filter1, stride=1)
print(conv1.shape)
pool1 = max_pool(conv1, pool_kernel, stride=2)
print(pool1.shape)
conv2 = conv(pool1, filter2, stride=1)
print(conv2.shape)
pool2 = max_pool(conv2, pool_kernel, stride=2)
print(pool2.shape)
flatten = torch.flatten(pool2,start_dim=1)
print(flatten.shape)

# Fully connected layers
z1 = flatten @ w1 + b1
af1 = relu(z1)
z2 = af1 @ w2 + b2
af2 = relu(z2)
z3 = af2 @ w3 + b3

# Softmax and Loss
z_max = z3.max(1, keepdim=True).values
norm_z = z3 - z_max
exp_norm_z = torch.exp(norm_z)
z_sum = exp_norm_z.sum(dim=1, keepdims=True)
z_sum_inv = z_sum**-1
probs = exp_norm_z * z_sum_inv
logprobs = probs.log()
loss = -logprobs[range(len(x_train)), y_train].mean()

torch.Size([5, 1, 26, 26])
torch.Size([5, 1, 13, 13])
torch.Size([5, 1, 11, 11])
torch.Size([5, 1, 5, 5])
torch.Size([5, 25])


In [1803]:
filter1.retain_grad()
conv1.retain_grad()
pool1.retain_grad()
filter2.retain_grad()
conv2.retain_grad()
pool2.retain_grad()
flatten.retain_grad()
z1.retain_grad()
af1.retain_grad()
z2.retain_grad()
af2.retain_grad()
z3.retain_grad()
z_max.retain_grad()
norm_z.retain_grad()
exp_norm_z.retain_grad()
z_sum.retain_grad()
z_sum_inv.retain_grad()
probs.retain_grad()
logprobs.retain_grad()

loss.backward()

In [1804]:
# Backward Atomic Softmax
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(len(x_train)), y_train] = -1.0 / len(x_train)
dprobs = (1.0 / probs) * dlogprobs
dz_sum_inv = (exp_norm_z * dprobs).sum(dim=1, keepdim=True)
dexp_norm_z = z_sum_inv * dprobs
dz_sum = (-z_sum**-2) * dz_sum_inv
dexp_norm_z += torch.ones_like(exp_norm_z) * dz_sum
dnorm_z = exp_norm_z * dexp_norm_z
dz_max = (-dnorm_z).sum(dim=1, keepdim=True)
dz3 = dnorm_z.clone()
dz_max = (-dnorm_z).sum(dim=1, keepdim=True)
dz3+=torch.nn.functional.one_hot(z3.max(1).indices,num_classes=z3.shape[1])*dz_max

# Backward Atomic Fully Connected Layer
dw3 = af2.T @ dz3
db3 = dz3.sum(0)
daf2 = dz3 @ w3.T
dz2 = z2.clone()
dz2 = torch.where(dz2 > 0, 1, 0).to(torch.float) * daf2
dw2 = af1.T @ dz2
db2 = dz2.sum(0)
daf1 = dz2 @ w2.T
dz1 = z1.clone()
dz1 = torch.where(dz1 > 0, 1, 0).to(torch.float) * daf1

# Backward Atomic CNN Layer
dflatten = dz1 @ w1.T
dpool2 = torch.ones_like(pool2) * dflatten.view(pool2.shape)
dconv2 = pool_back(conv2, pool_kernel, dpool2, stride=2)
dfilter2 = conv_back(pool1.clone(), dconv2, stride=1)
dpool1 = conv(dconv2 ,rotate_180(filter2), padding=2, stride=1)
dconv1 = pool_back(conv1, pool_kernel, dpool1, stride=2)
dfilter1 = conv_back(x_train, dconv1, stride=1)



In [1805]:
 #utitlity function to check manually calculated gradients with that of pytorch autograd
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

## Comparing gradients for 2 layers

In [1806]:
cmp('filter1', dfilter1, filter1)
cmp('conv1', dconv1, conv1)
cmp('pool1', dpool1, pool1)
cmp('filter2', dfilter2, filter2)
cmp('conv2', dconv2, conv2)
cmp('pool2', dpool2, pool2)
cmp('flatten', dflatten, flatten)
cmp('z1', dz1, z1)
cmp('af1', daf1, af1)
cmp('z2', dz2, z2)
cmp('af2', daf2, af2)
cmp('z3', dz3, z3)
cmp('z_max', dz_max, z_max)
cmp('norm_z', dnorm_z, norm_z)
cmp('exp_norm_z', dexp_norm_z, exp_norm_z)
cmp('z_sum', dz_sum, z_sum)
cmp('z_sum_inv', dz_sum_inv, z_sum_inv)
cmp('probs', dprobs, probs)
cmp('logprobs', dlogprobs, logprobs)


filter1         | exact: False | approximate: True  | maxdiff: 2.2737367544323206e-13
conv1           | exact: False | approximate: True  | maxdiff: 5.684341886080802e-14
pool1           | exact: False | approximate: True  | maxdiff: 5.684341886080802e-14
filter2         | exact: False | approximate: True  | maxdiff: 1.7053025658242404e-13
conv2           | exact: False | approximate: True  | maxdiff: 2.2737367544323206e-12
pool2           | exact: False | approximate: True  | maxdiff: 2.2737367544323206e-12
flatten         | exact: False | approximate: True  | maxdiff: 2.2737367544323206e-12
z1              | exact: False | approximate: True  | maxdiff: 4.3655745685100555e-11
af1             | exact: False | approximate: True  | maxdiff: 4.3655745685100555e-11
z2              | exact: False | approximate: True  | maxdiff: 4.656612873077393e-10
af2             | exact: False | approximate: True  | maxdiff: 4.656612873077393e-10
z3              | exact: False | approximate: True  | maxd